<table width="100%" style="padding:0; margin-left:-6px;">
    <tr style="padding=0;" width="100%">
        <td width="auto">
           <hr style="border:2px solid darkblue">
            <h1> nbX. Query for calibrator observations</h1>
           <hr style="border:2px solid darkblue">
        </td>
        <td width="60px">
           <img src="ALMAsmall.png" align="right"/> 
        </td>
    </tr>
</table>    

**This Notebook builds off of example notebook #2: nb2_ALMA_Query_public_catalogue.ipynb**

**Find more here: https://almascience.nrao.edu/alma-data/archive/archive-notebooks**

In this notebook functions are presented which can query a list of sources from a catalogue: 

* By ALMA source name
* By coordinate

For lists with a very large number of sources, run one query per source (or per block of e.g. 100 sources)

-------------

The relevant columns in the ALMA Archive datasets are 
* *s_ra*, *s_dec* (RAD and DEC in degress)
* *target_name* (source name)
 
--------- ----- -----


In [30]:
import astropy
import astropy.coordinates
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import pyvo
from datetime import datetime
import os



# service = pyvo.dal.TAPService("https://almascience.eso.org/tap")      # for the EU ALMA TAP service
# service = pyvo.dal.TAPService("https://almascience.nao.ac.jp/tap")  # for the EA ALMA TAP service
service = pyvo.dal.TAPService("https://almascience.nrao.edu/tap")   # for the NA ALMA TAP service

<hr style="border:2px solid darkblue"> </hr>


## Query list of alma source names
<hr style="border:2px solid darkblue"> </hr>



In [23]:
def query_alma_source_name_list_and_date(service, name_list, start_date, end_date):
    """queries for list of source names as saved in the ALMA system"""

    start_mjd = date_to_mjd(start_date)
    end_mjd = date_to_mjd(end_date)
    query = f"""  
            SELECT *  
            FROM ivoa.obscore  
            WHERE target_name in ('{"','".join(name_list)}') 
              AND t_min BETWEEN {start_mjd} AND {end_mjd}
            """
            #AND t_min BETWEEN ('{"','".join(start_date)}' AND '{"','".join(end_date)}') """
    return service.search(query).to_table().to_pandas()

In [18]:
def date_to_mjd(date_str):
    # Step 1: Parse date
    dt = datetime.strptime(date_str, "%Y-%m-%d")
    
    # Step 2: Convert to Julian Date using the algorithm
    year = dt.year
    month = dt.month
    day = dt.day
    
    if month <= 2:
        year -= 1
        month += 12

    A = year // 100
    B = 2 - A + A // 4

    jd = int(365.25 * (year + 4716)) + int(30.6001 * (month + 1)) + day + B - 1524.5
    
    # Step 3: Convert to MJD
    mjd = jd - 2400000.5
    return mjd


## Example a: Query a list of sources using the ALMA source name and observing period

** Note that it can take a VERY long time if you just query by sourcename because the calibrators were observed so many times

In [25]:
## Here I selected two known calibrators, and a time span of 1 year, 
## just to limit the search to a manageable amount.
## You can choose any sources and any timespan.
## You can find calibrators on https://almascience.nrao.edu/sc/

source_list = ['3C279', 'J1924-2914'] 
start_date = '2023-10-01'
end_date = '2024-10-01'

results = query_alma_source_name_list_and_date(service, source_list, start_date, end_date)

results.head(5)

,pol_states,obs_publisher_did,obs_collection,facility_name,instrument_name,obs_id,dataproduct_type,calib_level,collections,target_name,s_ra,s_dec,s_fov,s_region,s_xel1,s_xel2,em_xel,t_xel,pol_xel,s_resolution,t_min,t_max,t_exptime,t_resolution,em_min,em_max,em_res_power,sensitivity_10kms,cont_sensitivity_bandwidth,pwv,group_ous_uid,member_ous_uid,asdm_uid,obs_title,type,scan_intent,science_observation,spatial_scale_max,qa2_passed,bib_reference,science_keyword,scientific_category,pi_userid,pi_name,spectral_resolution,lastModified,o_ucd,access_url,access_format,access_estsize,proposal_id,data_rights,gal_longitude,gal_latitude,band_list,em_resolution,bandwidth,antenna_arrays,is_mosaic,obs_release_date,spatial_resolution,frequency_support,frequency,velocity_resolution,obs_creator_name,pub_title,first_author,authors,pub_abstract,publication_year,proposal_abstract,schedblock_name,proposal_authors
0,/XX/YY/,ADS/JAO.ALMA#2022.1.00875.L,ALMA,JAO,ALMA,uid://A001/X2d20/X1f23.source.J1924-2914.spw.39,cube,2,,J1924-2914,291.212733,-29.2417,0.005653,Polygon ICRS 291.214469 -29.244087 291.213340 ...,<NA>,<NA>,960,1,2,0.304964,60450.283274,60457.279893,3024.0,3024.0,0.001020,0.001020,1.041030e+06,1.235729,0.083302,1.14874,uid://A001/X2d20/X1f22,uid://A001/X2d20/X1f23,uid://A002/X1178b24/X811c,The ALMA Disk-Exoplanet C/Onnection,L,BANDPASS FLUX WVR,F,4.465697,T,,"Disks around low-mass stars, Exo-planets",Disks and planet formation,cleevesli,"Cleeves, Ilse",282.226562,2024-08-21T18:10:02.819,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid:/...,applicati,<NA>,2022.1.00875.L,Proprietary,9.344108,-19.606808,7,9.795880e-10,234375000.0,A001:DV12 A002:DA45 A007:DV20 A008:DV17 A010:D...,F,2025-07-24T18:09:54.000,0.304964,"[278.29..278.35GHz,141.11kHz,1.3mJy/beam@10km/...",293.891882,151.263775,ALMA,,,,,<NA>,Protoplanetary disks set the initial compositi...,J162616._a_07_TM1,"Yen, Hsi-Wei; Gonzalez, Camilo; Pegues, Jamila..."
1,/XX/YY/,ADS/JAO.ALMA#2022.1.00875.L,ALMA,JAO,ALMA,uid://A001/X2d20/X1f23.source.J1924-2914.spw.21,cube,2,,J1924-2914,291.212733,-29.2417,0.005653,Polygon ICRS 291.214469 -29.244087 291.213340 ...,<NA>,<NA>,480,1,2,0.304964,60450.283274,60457.279893,3024.0,3024.0,0.001075,0.001075,1.976623e+06,1.285930,0.083302,1.14874,uid://A001/X2d20/X1f22,uid://A001/X2d20/X1f23,uid://A002/X1178b24/X811c,The ALMA Disk-Exoplanet C/Onnection,L,BANDPASS FLUX WVR,F,4.465697,T,,"Disks around low-mass stars, Exo-planets",Disks and planet formation,cleevesli,"Cleeves, Ilse",141.113281,2024-08-21T18:10:02.819,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid:/...,applicati,<NA>,2022.1.00875.L,Proprietary,9.344108,-19.606808,7,5.439884e-10,58593750.0,A001:DV12 A002:DA45 A007:DV20 A008:DV17 A010:D...,F,2025-07-24T18:09:54.000,0.304964,"[278.29..278.35GHz,141.11kHz,1.3mJy/beam@10km/...",278.868509,151.263775,ALMA,,,,,<NA>,Protoplanetary disks set the initial compositi...,J162616._a_07_TM1,"Yen, Hsi-Wei; Gonzalez, Camilo; Pegues, Jamila..."
2,/XX/YY/,ADS/JAO.ALMA#2022.1.00875.L,ALMA,JAO,ALMA,uid://A001/X2d20/X1f23.source.J1924-2914.spw.23,cube,2,,J1924-2914,291.212733,-29.2417,0.005653,Polygon ICRS 291.214469 -29.244087 291.213340 ...,<NA>,<NA>,480,1,2,0.304964,60450.283274,60457.279893,3024.0,3024.0,0.001073,0.001073,1.981046e+06,1.284985,0.083302,1.14874,uid://A001/X2d20/X1f22,uid://A001/X2d20/X1f23,uid://A002/X1178b24/X811c,The ALMA Disk-Exoplanet C/Onnection,L,BANDPASS FLUX WVR,F,4.465697,T,,"Disks around low-mass stars, Exo-planets",Disks and planet formation,cleevesli,"Cleeves, Ilse",141.113281,2024-08-21T18:10:02.819,phot.flux.density;phys.polarization,https://almascience.org/datalink/sync?ID=uid:/...,applicati,<NA>,2022.1.00875.L,Proprietary,9.344108,-19.606808,7,5.415619e-10,58593750.0,A001:DV12 A002:DA45 A007:DV20 A008:DV17 A010:D...,F,2025-07-24T18:09:54.000,0.304964,"[278.29..278.35GHz,141.11kHz,1.3mJy/beam@10km/...",279.492560,151.263775,ALMA,,,,,<NA>,Protoplanetary disks set the 

In [28]:
## prepare to download
mousids = np.unique(results['member_ous_uid'])
print('Found {} unique MOUS'.format(mousids.size))

Found 462 unique MOUS


In [43]:
from pyvo.dal.adhoc import DatalinkResults

# retrieving the first dataset (see the index [0]) here as an example
datalink = DatalinkResults.from_result_url(f"https://almascience.nrao.edu/datalink/sync?ID={mousids[0]}")
urls = datalink['access_url']
urls

masked_array(data=['https://almascience.nrao.edu/dataPortal/member.uid___A001_X14ed_X1e7.README.txt',
                   'https://almascience.nrao.edu/dataPortal/2019.2.00120.S_uid___A001_X14ed_X1e7_001_of_001.tar',
                   '',
                   'https://almascience.nrao.edu/dataPortal/2019.2.00120.S_uid___A001_X14ed_X1e7_auxiliary.tar',
                   '',
                   'https://almascience.nrao.edu/dataPortal/2019.2.00120.S_uid___A002_X10e318c_X1555.asdm.sdm.tar',
                   'https://almascience.nrao.edu/dataPortal/2019.2.00120.S_uid___A002_X1103e51_X5d35.asdm.sdm.tar'],
             mask=[False, False, False, False, False, False, False],
       fill_value=array('?', dtype='<U1'),
            dtype=object)

In [45]:
## IF you want to download the raw data, run this cell
download = True
if download: 
    print('Downloading raw files')
    
    for dl in datalink:    
        my_url = dl['access_url']
        if my_url.endswith('.asdm.sdm.tar'):
            print(my_url)
            dl.cachedataset(filename=os.path.basename(dl['access_url']))
else: 
    print('If you want to download files, set download = True')

https://almascience.nrao.edu/dataPortal/2019.2.00120.S_uid___A002_X10e318c_X1555.asdm.sdm.tar
https://almascience.nrao.edu/dataPortal/2019.2.00120.S_uid___A002_X1103e51_X5d35.asdm.sdm.tar
